# Numba

**Python** is inherently slow:

1. It interpret the code line by line (no compiling)
1. It runs the code line by line (serial)

**Numba** package allow for simple **just-in-time compilation**. 

Just add decorator `numba.njit` on top of a function.

1. First run is slower because code is analyzed.
1. But subsequent calls are then a lot faster. 

*The input types can, however, not change between calls because numba infer them on the first call.*

**Table of contents**<a id='toc0_'></a>    
- 1. [Numpy](#toc1_)    
- 2. [Numba](#toc2_)    
- 3. [Paralization](#toc3_)    
- 4. [Calling an optimizer](#toc4_)    
- 5. [Limitation](#toc5_)    
  - 5.1. [EconModelClass](#toc5_1_)    
- 6. [Summary](#toc6_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [1]:
import numpy as np
import numba as nb

## 1. <a id='toc1_'></a>[Numpy](#toc0_)

Basic implementation:

In [2]:
def myfun_numpy(x1,x2):

    y = np.empty(x1.size)

    for i in range(x1.size):
        if x1[i] < 0.5:
            y[i] = np.sum(np.exp(x2*x1[i]))
        else:
            y[i] = np.sum(np.log(x2*x1[i]))

    return y

Vectorized implementation:

In [3]:
def myfun_numpy_vec(x1,x2):

    y = np.empty((1,x1.size))
    I = x1 < 0.5

    y[I] = np.sum(np.exp(x2*x1[I]),axis=0)
    y[~I] = np.sum(np.log(x2*x1[~I]),axis=0)
    
    return y

Inputs:

In [4]:
rng = np.random.default_rng(1234)
x1 = rng.uniform(size=10**6)
x2 = rng.uniform(size=np.int64(100))
x1_np = x1.reshape((1,x1.size))
x2_np = x2.reshape((x2.size,1))

Same results:

In [5]:
assert np.allclose(myfun_numpy_vec(x1_np,x2_np),myfun_numpy(x1,x2))

Timing:

In [6]:
%timeit myfun_numpy(x1,x2)

7.78 s ± 446 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%timeit myfun_numpy_vec(x1_np,x2_np)

1.07 s ± 57 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## 2. <a id='toc2_'></a>[Numba](#toc0_)

In [8]:
@nb.njit
def myfun_numba(x1,x2):

    y = np.empty(x1.size)

    for i in range(x1.size):
        
        if x1[i] < 0.5:
            y[i] = np.sum(np.exp(x2*x1[i]))
        else:
            y[i] = np.sum(np.log(x2*x1[i]))
            
    return y

# call to just-in-time compile
%time myfun_numba(x1,x2)

# actual measurement
%timeit myfun_numba(x1,x2)

assert np.allclose(myfun_numpy_vec(x1_np,x2_np),myfun_numba(x1,x2))

CPU times: total: 1.3 s
Wall time: 3.01 s
797 ms ± 28.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


You can also call the Python-version:

In [9]:
%time myfun_numba.py_func(x1,x2);

CPU times: total: 3.38 s
Wall time: 7.56 s


**Caveats:** Only a limited number of Python and Numpy features are supported inside just-in-time compiled functions.

- [Supported Python features](https://numba.pydata.org/numba-doc/dev/reference/pysupported.html)
- [Supported Numpy features](https://numba.pydata.org/numba-doc/dev/reference/numpysupported.html)


## 3. <a id='toc3_'></a>[Paralization](#toc0_)

In [10]:
@nb.njit(parallel=True)
def myfun_numba_par(x1,x2):

    y = np.empty(x1.size)
    
    for i in nb.prange(x1.size): # in parallel across threads
        if x1[i] < 0.5:
            y[i] = np.sum(np.exp(x2*x1[i]))
        else:
            y[i] = np.sum(np.log(x2*x1[i]))
            
    return y

assert np.allclose(myfun_numpy_vec(x1_np,x2_np),myfun_numba_par(x1,x2))

myfun_numba_par(x1,x2)
%timeit myfun_numba_par(x1,x2)

121 ms ± 17.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## 4. <a id='toc4_'></a>[Calling an optimizer](#toc0_)

Using solver from `QuantEcon` (see [documentation](https://quanteconpy.readthedocs.io/en/latest/index.html)).

`scipy` solvers can not be used in `numba`.

In [11]:
#%pip instal quantecon

In [12]:
import quantecon as qe

In [13]:
n = 4000
alphas = rng.uniform(size=n)
betas = rng.uniform(size=n)
gammas = rng.uniform(size=n)

In [14]:
@nb.njit
def solver_nb(alpha,beta,gamma):

    def obj(x,alpha,beta,gamma):
        return (x[0]-alpha)**2 + (x[1]-beta)**2 + (x[2]-gamma)**2

    res = qe.optimize.nelder_mead(obj,np.array([0.0,0.0,0.0]),args=(alpha,beta,gamma))

    return res.x

**Serial version:**

In [15]:
@nb.njit
def serial_solver_nb(alphas,betas,gammas):

    n = alphas.size
    xopts = np.zeros((n,3))

    for i in range(n):
        xopts[i,:] = solver_nb(alphas[i],betas[i],gammas[i])

%time serial_solver_nb(alphas,betas,gammas)
%time serial_solver_nb(alphas,betas,gammas)

CPU times: total: 8.67 s
Wall time: 17.9 s
CPU times: total: 2.55 s
Wall time: 3.38 s


**Parallel version:**

In [16]:
@nb.njit(parallel=True)
def parallel_solver_nb(alphas,betas,gammas):

    n = alphas.size
    xopts = np.zeros((n,3))

    for i in nb.prange(n):
        xopts[i,:] = solver_nb(alphas[i],betas[i],gammas[i])

%time parallel_solver_nb(alphas,betas,gammas)
%time parallel_solver_nb(alphas,betas,gammas)

CPU times: total: 9.98 s
Wall time: 6.06 s
CPU times: total: 7.08 s
Wall time: 1.01 s


## 5. <a id='toc5_'></a>[Limitation](#toc0_)

Consider a simple function:

In [17]:
@nb.njit
def myfunc(par,x):
    return par.alpha + x 

`SimpleNamespace` is not supported by `numba`:

In [18]:
from types import SimpleNamespace

par = SimpleNamespace()
par.alpha = 0.5
x = 10.0

try:
    myfunc(par,x)
except Exception as e:
    print('does not work')

does not work


**Simple solution:** Use `namedtuple` instead.

In [19]:
from collections import namedtuple

ParClass = namedtuple('ParClass',par.__dict__.keys())
par_ = ParClass(**par.__dict__)

myfunc(par_,x)

10.5

**My solution:** Use the `EconModelClass` in my `EconModel` package. Simple introduction [here](https://github.com/NumEconCopenhagen/AdvMacroHet/blob/main/01-Introduction/04.%20EconModelClass.ipynb).

Also functionalities for copying, saving and loading. (And interfacte to fast C++ code.)

### 5.1. <a id='toc5_1_'></a>[Extra: EconModelClass](#toc0_)

In [20]:
#%pip install EconModel

In [21]:
from EconModel import EconModelClass, jit 

In [22]:
class MyModelClass(EconModelClass):
    
    # __init__ is inherited from EconModelClass
    
    def settings(self): # required
        """ choose settings """
            
        pass # nothing chosen here
    
    def setup(self): # required
        """ set free parameters """
        
        par = self.par
        
        par.N = 10
        par.a = 2.0
        par.b = 1.0
        par.threads = 4
        par.txt = 'a'
        par.txtlist = 'N|threads'
        
    def allocate(self): # required
        """ set compound parameters and allocate arrays """
        
        par = self.par
        sol = self.sol
        
        par.X = np.linspace(0,10,par.N)
        sol.Y = np.zeros(par.N)
        par.Z = np.ones(par.N,dtype=int)
    
    def solve(self): # user-defined
        """ solve the model"""
        
        par = self.par
        sol = self.sol
        
        sol.Y[:] = par.X*(par.a+par.b)*par.Z

Create model (calls `settings`, `setup` and `allocate`):

In [23]:
model = MyModelClass(name='example')

Solution in model:

In [24]:
@nb.njit
def fun(par,sol):
    sol.Y[:] = par.X*(par.a+par.b)

In [25]:
def test_numba(model):
    with jit(model) as model_jit: # automatically convert to namedtuple for numba
        fun(model_jit.par,model_jit.sol)

In [26]:
%time test_numba(model)
%time test_numba(model)

CPU times: total: 297 ms
Wall time: 402 ms
CPU times: total: 0 ns
Wall time: 175 μs


Check solution:

In [27]:
assert np.allclose(model.par.X*(model.par.a+model.par.b),model.sol.Y)

## 6. <a id='toc6_'></a>[Summary](#toc0_)

Using `numba` to speed-up your code is straighforward.

Still not fast enough? You probably need to lead C++, which can be called form Python using `EconModelClass`.